## 安装，配置使用 CMake 工具链

按照 [官方教程](https://github.com/ngtcp2/ngtcp2) 的前一半安装好 `OpenSSL_1_1_1q+quic` 和 `nghttp3`

``` bash
$ git clone --depth 1 -b OpenSSL_1_1_1q+quic https://github.com/quictls/openssl
$ cd openssl
$ # For Linux
$ ./config enable-tls1_3 --prefix=$PWD/build
$ make -j$(nproc)
$ make install_sw
$ cd ..
$ git clone https://github.com/ngtcp2/nghttp3
$ cd nghttp3
$ autoreconf -i
$ ./configure --prefix=$PWD/build --enable-lib-only
$ make -j$(nproc) check
$ make install
$ cd ..
$ git clone https://github.com/ngtcp2/ngtcp2
$ cd ngtcp2
```

在 VSCode 中配置 CMake ，重点配置 CMake 的环境变量 PKG_CONFIG_PATH 为刚刚安装好的两个库，`settings.json` 如下

``` json
{
    "cmake.configureOnOpen": true,
    "cmake.environment": {
        "PKG_CONFIG_PATH": "${workspaceFolder}/../openssl/build/lib/pkgconfig:${workspaceFolder}/../nghttp3/build/lib/pkgconfig",
    },
    "files.associations": {
        "string_view": "cpp"
    },
}
```

即可使用 CMake 编译项目

## 使用 examples/ 中的客户端和服务端

`examples/server.cc` 似乎实现了一个使用 http3 的服务器，可以使用 `examples/client.cc` 连接与访问

默认情况下编译好的可执行文件在 `build/` 中对应目录下

### 服务端

先用 OpenSSL 签发证书，参考：<https://zhuanlan.zhihu.com/p/367630416>

生成 PRIVATE_KEY_FILE 为 `server.key`，CERTIFICATE_FILE 为 `server.crt`

再生成一些用于传输的文件，比如生成一个 1M 大小的全零文件 `1M.file`

``` bash
dd if=/dev/zero of=1M.file bs=1M count=1
```

如下命令运行服务端

- '*' 表示监听所有地址
- 2333 为端口
- `--qlog-dir` 为 qlog 日志目录，目录下文件名为 Source Connection ID，qlog 日志文件可以在这个网站上可视化: https://qvis.quictools.info/#/files
- `-q` 关闭调试输出（原程序中调试信息过多），可以修改源码直接输出需要的信息

In [ ]:
!./build/examples/server '*' 2333 ./build/examples/server.key ./build/examples/server.crt --qlog-dir ./build/examples/server_log/ -q

### 客户端

- `--download` 为文件下载目录
- 末尾为要访问的 URL

In [ ]:
!./build/examples/client 127.0.0.1 2333  --qlog-dir ./build/examples/client_log/ --download ./build/examples/download/ https://127.0.0.1:2333/build/examples/data/1M.file

## 使用 Mahimahi 控制网络条件

[Mahimahi](http://mahimahi.mit.edu/)

- `mm-delay` 设置延迟，如 `mm-delay 100` 设置 100ms 延迟
- `mm-loss` 设置丢包率，如 `mm-loss uplink 0.2 mm-loss downlink 0.3` 设置上行 20% 丢包率，下行 30% 丢包率
- `mm-link` 使用 trace 文件模拟网络速率，如 `mm-link /usr/share/mahimahi/traces/Verizon-LTE-short.up /usr/share/mahimahi/traces/Verizon-LTE-short.down` 使用对应的上下行 trace 文件。文档里没说的是 `mm-link` 还有参数可限制缓冲区大小： `--downlink-queue=droptail --downlink-queue-args=bytes=???`

Mahimahi 仓库也提供了一些实际网络测速得到的 trace 文件：<https://github.com/ravinet/mahimahi/tree/master/traces>

关于 trace 文件：

> Each line gives a timestamp in milliseconds (from the beginning of the
trace) and represents an opportunity for one 1500-byte packet to be
drained from the bottleneck queue and cross the link. If more than one
MTU-sized packet can be transmitted in a particular millisecond, the
same timestamp is repeated on multiple lines.

> A fixed-rate link may also be represented a trace file. For example, a
link that can pass one MTU-sized packet per millisecond (12 Mbps) can
be represented by a file that contains just "1". By default, mm-link
will repeat the trace file when it reaches the end, so this trace
would represent one packet per millisecond.

这些指令可以结合使用，相当于多层包装，例如：

``` bash
mm-delay 100 mm-loss uplink 0.2 mm-loss downlink 0.2 mm-link /usr/share/mahimahi/traces/Verizon-LTE-short.up /usr/share/mahimahi/traces/Verizon-LTE-short.down
```

在 Mahimahi container 中，访问本机架设的服务需要使用 "base" container 的 IP 变量 $MAHIMAHI_BASE 作为访问的地址，使用 127.0.0.1 访问的是当前容器。客户端使用可以改成如下命令：

``` bash
./build/examples/client $MAHIMAHI_BASE 2333  --qlog-dir ./build/examples/client_log/ --download ./build/examples/download/ https://$MAHIMAHI_BASE:2333/build/examples/data/1M.file
```

## 20220718 实验

初步熟悉代码，在源码中对 CUBIC 初步修改，直接记录上一次连接最后的 CWnd，在下一次连接时直接使用。

服务端：

``` bash
./build/examples/server '*' 2333 ./build/examples/server.key ./build/examples/server.crt --qlog-dir ./build/examples/server_log/ -q
```

客户端：

``` bash
mm-delay 100 mm-link --downlink-queue=droptail --downlink-queue-args=bytes=??? /usr/share/mahimahi/traces/Verizon-LTE-short.up /usr/share/mahimahi/traces/Verizon-LTE-short.down
```

``` bash
./build/examples/client $MAHIMAHI_BASE 2333 --download ./build/examples/download/ https://$MAHIMAHI_BASE:2333/build/examples/data/1M.file -q
```

## 20220801 实验

### 目标

- [x] 将修改后的拥塞控制算法提取为单独的文件（添加 --cc 可指定的拥塞控制算法 `scubic`）
- [x] CUBIC 初始化时即设置 CWnd（为 CUBIC 初始化函数添加参数 `ngtcp2_conn_stat *cstat`）
- [x] 解析连接 IP 地址，建立 Hash 表（添加参数 `const ngtcp2_path *path`，从中解析 IP 地址；Hash 表暂时在文件中以 static 数组形式存储，Hash 函数暂时为取 IP 地址前 10 位）
- [x] 客户端检测数据传输完成后终止连接（有参数 `--exit-on-all-streams-close`）
- [x] 命令行指定测试参数/自动化测试脚本

### 实验内容

#### 服务端

``` bash
./build/examples/server '*' 2333 ./run/data/server.key ./run/data/server.crt --qlog-dir ./run/logs/server_log/ -q --cc scubic
```

mm-delay 100 mm-link ./traces/1.0.trace ./traces/1.0.trace

#### Trace 生成

使用 `python3 ./run/gen_traces.py [multiple]` 生成 `0.1.trace` 至 `2.0.trace`，其中 `multiple` 设定目标带宽带宽为 12Mbps 的多少倍，生成的 20 个 trace 文件为 0.1 至 2.0 倍目标带宽。

#### 客户端

使用 `--exit-on-all-streams-close` 参数在完成请求时断开连接。

将 Mahimahi 的创建和客户端的使用合为一个指令便于脚本化，sleep 是用于防止服务端无法正常关闭连接：

```bash
mm-delay 100 sh -c './build/examples/client $MAHIMAHI_BASE 2333 https://$MAHIMAHI_BASE:2333/run/data/1M.file -q --exit-on-all-streams-close && sleep 1'
```

然而再加上 mm-link 就无法执行了，似乎 mm-link 对命令的解析有问题

于是把客户端的运行单独作为一个脚本 `run_client.sh`：

```bash
#!/bin/bash

./build/examples/client $MAHIMAHI_BASE 2333 https://$MAHIMAHI_BASE:2333/run/data/1M.file -q --exit-on-all-streams-close
sleep 1
```

这样就可以一个命令运行客户端：

```
mm-delay 100 mm-link --downlink-queue=droptail --downlink-queue-args=bytes=300000 ./run/traces/1.0.trace ./run/traces/1.0.trace ./run/run_client.sh
```

#### 测试脚本

见 `./run/run.sh`，另外 `./run/collect.py` 可以用于收集数据绘图分析。

## 20220804 实验

### 上一次实验中需要解决的问题

- [x] 更改带宽后 buffer size 也需要改为对应的 BDP（测试脚本中修改了对应参数）
- [x] 统计数据时统计丢包情况（统计了丢包数量，暂未统计字节数量）
- [x] 降低 RTT（使用 20ms, 50ms 测试）
- [x] 增加发送文件大小从而更加充分地展现拥塞控制过程（测试脚本增加对应参数，文件大小增加至多少未定）
- [x] 实现 Stateful-TCP 论文中提到的 *"bypass Slow-Start"* （实现细节有区别，见附录）
- [x] 实现 Stateful-TCP 论文中的 *"bandwidth estimation"* （GCBE 带宽估计算法，见附录）
- [x] 实现 Pacing

### 问题

- 统计丢包字节
- 最终 Bytes in Flight 非零
- RWnd / AWnd ?
- min_RTT 的估计，是否有影响
- Mahimahi 设定 delay 后，实际 RTT 会略大于 2*delay (高约 0.5 s )，是否需要补偿
- 目前实现的 GCBE 带宽估计算法有时会出现较为严重的带宽高估（高估约 1.5~2 倍），可能出现的原因是 `gap-compensated` 后剩余的样本时间较小（RTT 20 ms，只剩 2~3 ms），解决方案可以尝试不对这种样本进行 `gap-compensated` ，或者缩小带宽缓存池大小（现同论文为 5）
- 高带宽情形下初始发包数量似乎无法达到设定的 Cwnd, 可能的原因有：发包速率有限（低带宽下是可以达标的），但是实测发包数量达到 270000 左右字节后会进入停顿；有另外的参数限制窗口，比如 RWnd 之类，但未在源码中搜索到 RWnd/AWnd 等关键词。 `ngtcp2_conn.c` 中 13327 行 `ngtcp2_conn_update_pkt_tx_time` 函数为 pacing 发包时更新下次发包时间的函数，在其中进行输出可供调试 pacing 发包。

### 实验数据总结

#### 1M

- mul <= 0.5 之后 SCUBIC 的 FCT (Flow Completion Time) 无显著优势，且开始出现显著的丢包情况，越小越明显
- 0.5 < mul <= 1.0 时可以避免常规 CUBIC 的慢启动阶段末尾带来的丢包
- mul >= 1.4 之后 SCUBIC 的 FCT 要更长，因为缺失了慢启动阶段探测的带宽

#### 128K

- mul <= 0.3 有如上劣势
- mul 在 2.0 以内 SCUBIC 都有优势

### 设计

#### 初始 pacing 阶段若发生丢包/RTT升高则？

尝试通过初始几个 RTT 估计带宽，按估计带宽发包

#### 若 RTT 不变（带宽不变/升高）则？

重新进入慢启动阶段，按照 BBR 的速率进行增窗（因为引入了 pacing），重点关注如何通过 RTT 可靠地判断带宽升高，按 CUBIC 退出慢启动，考虑 BBR 排空方式？

#### RTT 噪声？

暂不考虑

## 20220829 实验 - 开始对 SCUBIC 做改进

### 目标

- [x] 固定当前的 SCUBIC，新建拥塞控制算法 SCUBIC2
- [x] 通过初始几个 RTT 估计带宽
- [x] 参考上次实验的设计进行实现
- [ ] 改进 GCBE，抑制带宽高估

### 实验结果

#### 估计带宽

实验中发现，即使是 pacing 发包，也会有不少包在同一个时间戳上发出，导致 RTT 测量出现递增，因此尝试使用 ACK 估计带宽。

使用前几个 ACK 的 recv_ts 和 bytes_delivered 估计带宽，在带宽变小达到一定程度时能够较好地估计（大概 3~4 个 ACK 即可达到 ±10% 左右），在带宽不变或变大时易低估。

#### 带宽变化判断

记 $CWnd$ 为当前拥塞窗口，$CWnd_{preferred}$ 为使用估计带宽和 min_rtt 算得的理想拥塞窗口

##### 第一种

比较 $CWnd_{preferred}$ 和 $0.8 \times CWnd$

< 则令 $ssthresh = CWnd = CWnd_{preferred}$，进入排空阶段

\> 则重新进入慢启动

见 `data20220827/`

##### 第二种

比较 $2 \times CWnd_{preferred}$ 和 $CWnd$

< 则令 $ssthresh = CWnd = 2 \times CWnd_{preferred}$，进入排空阶段

\> 则重新进入慢启动

见 `data20220827-2/`

#### 排空阶段

记录进入该阶段的时间戳，对于该时间戳前发送的包，收到 ACK 不会增加 CWnd，丢包不会降低 CWnd。

当收到该时间戳之后发送的包的 ACK 时，该阶段结束。结束该阶段不影响对上述 ACK 的响应策略。

#### 重新进入慢启动

与正常慢启动无异

#### 对比测量

带宽降低的情况下，对比 SCUBIC 流完成时间基本都是变长（更劣）

带宽升高，会更优，因为可以快速上探带宽

### 问题

- 初始握手包的 ACK 会使 CWnd 略微增加，暂未修正
- 每次连接的带宽测定具有随机性，因此对比实验中需要手动设定前一次连接的状态
- 一端记录的 Bytes in Flight 似乎不能直接反映路径中的带宽利用情况，这似乎也导致了 CWnd 设置为 BDP 不能充分利用带宽

### 想法

pacing 可以减少丢包，不 pacing 可以便于估计带宽，可以尝试前几个包不 pacing 用于估计带宽。

一种想法是先发按默认 CWnd 连续发包，获得测速结果后判断是否使用先前的状态

## 附录

### SCUBIC 与原论文在实现细节上的区别

- 关于 *"bypass Slow-Start"*: 目前发现服务端在收到客户端请求后，第一个 RTT 内只会发送 1600 字节内容，分别为 `Initial Space` 1 个包 166 字节，`Handshake Space` 2 个包 1034 + 336 = 1370 字节，`Application Space` 1 个包 64 字节。所以目前的实现是将 `Application Space` 中的第二个包（pkt_num >= 1）的 ACK 作为 Stateful-TCP 论文中 `STARTUP PHASE` 的结束点，在收到该 ACK 后退出慢启动，将 CWnd 设为 Bytes in flight。检查 log 时又发现 `Application Space` 中第一个包的 ACK 会导致 delivery_rate_sec 出现统计数据，从而 target_cwnd 有值，此时 target_cwnd 会对发包有限制，现已将对应代码注释掉。
- 关于 *"bandwidth estimation"*: 在一个 `measurement cycle` 内若只有一个 ACK, 原论文中未定义此时行为，目前的实现是使用这一个 ACK 计算带宽，即不做论文中的操作
- 哈希表大小，论文中取 20 位，目前的实现取了 10 位